# GoogleNet

paper: https://arxiv.org/pdf/1409.4842.pdf

<ul>
    <li> All the convolutions use ReLu </li>
    <li> Inception Blocks are standardized made out of 4 differnet streams </li>
    <li> Network starts with two ConvMaxPool blocks </li>
    <li> followed by a series of Inception Blocks separated by max Pool layers</li>
    <li> Fully Connected layer at the end </li>
<ul>

In [3]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Concatenate, AvgPool2D, Dropout, Flatten, Dense

**Inception Module**:
![alt text](data/inception_module.png "Inception Module")

In [10]:
def inception_block(x, filters):
    output_1 = Conv2D(filters[0], kernel_size=1, activation='relu')(x)
    
    output_2 = Conv2D(filters[1], kernel_size=1, activation='relu')(x)
    output_2 = Conv2D(filters[2], kernel_size=3, padding='same', activation='relu')(output_2)
    
    output_3 = Conv2D(filters[3], kernel_size=1, activation='relu')(x)
    output_3 = Conv2D(filters[4], kernel_size=5, padding='same', activation='relu')(output_3)
    
    output_4 = MaxPool2D(pool_size=3, strides=1, padding='same')(x)
    output_4 = Conv2D(filters[5], kernel_size=1, activation='relu')(output_4)
    
    output = Concatenate()([output_1, output_2, output_3, output_4])
    
    return output

In [11]:
input_layer = Input(shape=(224, 224, 3))
x = Conv2D(filters=64, kernel_size=7, strides=2, padding='same',activation='relu')(input_layer)
x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)

x = Conv2D(filters=64, kernel_size=1, activation='relu')(x)
x = Conv2D(filters=192, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)

x = inception_block(x, [64, 96, 128, 16, 32, 32]) #inception 3a
x = inception_block(x, [128, 128, 192, 32, 96, 64]) #inception 3b
x= MaxPool2D(pool_size=3, strides=2, padding='same')(x)

x = inception_block(x, [192, 96, 2018, 16, 48, 64]) #inception 4a
x = inception_block(x, [160, 112, 224, 24, 64, 64]) #inception 4b
x = inception_block(x, [128, 128, 256, 24, 64, 64]) #inception 4c
x = inception_block(x, [112, 144, 288, 32, 64, 64]) #inception 4d
x = inception_block(x, [64, 96, 128, 16, 32, 32]) #inception 4e
x= MaxPool2D(pool_size=3, strides=2, padding='same')(x)

x = inception_block(x, [256, 160, 320, 32, 128, 128]) #inception 5a
x = inception_block(x, [384, 192, 384, 48, 128, 128]) #inception 5b
x= AvgPool2D(pool_size=7, strides=1)(x)

x = Dropout(rate=0.4)(x)
output = Dense(units=1000, activation='softmax')(x)


In [ ]:
from tensorflow.keras import Model